# Algoritmo de  

In [11]:
import json
from itertools import permutations

def cargar_datos(json_data):
    """Carga la estructura JSON."""
    return json.loads(json_data)

In [12]:
import json
import pulp

def cargar_datos(json_data):
    """Carga la estructura JSON."""
    return json.loads(json_data)

def optimizar_horarios(data):
    """Optimiza la asignación de horarios usando programación lineal con PuLP."""
    problema = pulp.LpProblem("Optimización_de_Horarios", pulp.LpMaximize)
    
    empleados = {}  # Diccionario para almacenar los empleados y sus horarios
    variables = {}  # Variables de decisión
    
    for sucursal, dias in data.items():
        for dia, turnos in dias.items():
            for turno_tipo in ["T8d", "T7m"]:
                for turno in turnos.get(turno_tipo, []):
                    clave = (dia, turno["Hora_entrada"], turno["Hora_salida"])
                    var = pulp.LpVariable(f"x_{dia}_{turno['Hora_entrada']}_{turno['Hora_salida']}", cat='Binary')
                    variables[clave] = var
                    problema += var  # Maximizar la cantidad de turnos asignados
    
    # Restricción: Cada empleado debe trabajar exactamente 6 días
    for empleado_id in range(1, len(variables) // 6 + 1):
        nombre_empleado = f"vendedor_{empleado_id}"
        empleados[nombre_empleado] = {"role": "vendedor", "horarios": [], "horas": 0}
        dias_asignados = [var for clave, var in variables.items()]
        problema += pulp.lpSum(dias_asignados) == 6  # Exactamente 6 días
    
    # Restricción: No se permite miércoles libre
    for clave, var in variables.items():
        if "Miércoles" in clave[0]:
            problema += var == 1  # Todos deben trabajar el miércoles
    
    # Resolver el problema de optimización
    problema.solve()
    
    # Construir salida
    for clave, var in variables.items():
        if pulp.value(var) == 1:
            dia, entrada, salida = clave
            empleado = next(iter(empleados.keys()))
            empleados[empleado]["horarios"].append({"dia": dia, "hora_entrada": entrada, "hora_salida": salida})
            empleados[empleado]["horas"] += (salida - entrada)
    
    return empleados




In [13]:
# Cargar y procesar el JSON
ejemplo_json = './solucion.json'
data = {
    "10": {
        "10-02-2025 Lunes": {
            "T8d": [
                {
                    "Hora_entrada": 7,
                    "Hora_salida": 15,
                    "empleados": 3,
                    "empleado": "Vendedores",
                    "id": 1
                },
                {
                    "Hora_entrada": 10,
                    "Hora_salida": 18,
                    "empleados": 1,
                    "empleado": "Vendedores"
                }
            ],
            "T7m": [
                {
                    "Hora_entrada": 12,
                    "Hora_salida": 19,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 13,
                    "Hora_salida": 20,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 14,
                    "Hora_salida": 21,
                    "empleados": 2,
                    "empleado": "Vendedores"
                }
            ],
            "T6n": [],
            "Personal_necesario": {
                "7": 1,
                "8": 1,
                "9": 4,
                "10": 4,
                "11": 4,
                "12": 6,
                "13": 6,
                "14": 5,
                "15": 5,
                "16": 5,
                "17": 5,
                "18": 4,
                "19": 3,
                "20": 3
            },
            "Empleados_asignados": {
                "7": 3.0,
                "8": 3.0,
                "9": 3.0,
                "10": 4.0,
                "11": 4.0,
                "12": 5.0,
                "13": 6.0,
                "14": 8.0,
                "15": 5.0,
                "16": 5.0,
                "17": 5.0,
                "18": 4.0,
                "19": 3.0,
                "20": 2.0
            }
        },
        "11-02-2025 Martes": {
            "T8d": [
                {
                    "Hora_entrada": 7,
                    "Hora_salida": 15,
                    "empleados": 1,
                    "empleado": "Vendedores",
                    "id": 1
                },
                {
                    "Hora_entrada": 8,
                    "Hora_salida": 16,
                    "empleados": 2,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 11,
                    "Hora_salida": 19,
                    "empleados": 1,
                    "empleado": "Vendedores"
                }
            ],
            "T7m": [
                {
                    "Hora_entrada": 12,
                    "Hora_salida": 19,
                    "empleados": 2,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 14,
                    "Hora_salida": 21,
                    "empleados": 1,
                    "empleado": "Vendedores"
                }
            ],
            "T6n": [],
            "Personal_necesario": {
                "7": 1,
                "8": 1,
                "9": 4,
                "10": 3,
                "11": 4,
                "12": 6,
                "13": 6,
                "14": 5,
                "15": 5,
                "16": 4,
                "17": 4,
                "18": 4,
                "19": 3,
                "20": 1
            },
            "Empleados_asignados": {
                "7": 1.0,
                "8": 3.0,
                "9": 3.0,
                "10": 3.0,
                "11": 4.0,
                "12": 6.0,
                "13": 6.0,
                "14": 7.0,
                "15": 6.0,
                "16": 4.0,
                "17": 4.0,
                "18": 4.0,
                "19": 1.0,
                "20": 1.0
            }
        },
        "12-02-2025 Mi\u00e9rcoles": {
            "T8d": [
                {
                    "Hora_entrada": 7,
                    "Hora_salida": 15,
                    "empleados": 2,
                    "empleado": "Vendedores",
                    "id": 1
                },
                {
                    "Hora_entrada": 9,
                    "Hora_salida": 17,
                    "empleados": 2,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 11,
                    "Hora_salida": 19,
                    "empleados": 1,
                    "empleado": "Vendedores"
                }
            ],
            "T7m": [
                {
                    "Hora_entrada": 14,
                    "Hora_salida": 21,
                    "empleados": 2,
                    "empleado": "Vendedores"
                }
            ],
            "T6n": [],
            "Personal_necesario": {
                "7": 1,
                "8": 1,
                "9": 4,
                "10": 4,
                "11": 5,
                "12": 6,
                "13": 5,
                "14": 4,
                "15": 4,
                "16": 5,
                "17": 4,
                "18": 4,
                "19": 2,
                "20": 2
            },
            "Empleados_asignados": {
                "7": 2.0,
                "8": 2.0,
                "9": 4.0,
                "10": 4.0,
                "11": 5.0,
                "12": 5.0,
                "13": 5.0,
                "14": 7.0,
                "15": 5.0,
                "16": 5.0,
                "17": 3.0,
                "18": 3.0,
                "19": 2.0,
                "20": 2.0
            }
        },
        "13-02-2025 Jueves": {
            "T8d": [
                {
                    "Hora_entrada": 7,
                    "Hora_salida": 15,
                    "empleados": 2,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 8,
                    "Hora_salida": 16,
                    "empleados": 1,
                    "empleado": "Vendedores"
                }
            ],
            "T7m": [
                {
                    "Hora_entrada": 11,
                    "Hora_salida": 18,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 14,
                    "Hora_salida": 21,
                    "empleados": 3,
                    "empleado": "Vendedores"
                }
            ],
            "T6n": [],
            "Personal_necesario": {
                "7": 2,
                "8": 2,
                "9": 4,
                "10": 3,
                "11": 4,
                "12": 7,
                "13": 5,
                "14": 5,
                "15": 4,
                "16": 4,
                "17": 4,
                "18": 3,
                "19": 3,
                "20": 3
            },
            "Empleados_asignados": {
                "7": 2.0,
                "8": 3.0,
                "9": 3.0,
                "10": 3.0,
                "11": 4.0,
                "12": 4.0,
                "13": 4.0,
                "14": 7.0,
                "15": 5.0,
                "16": 4.0,
                "17": 4.0,
                "18": 3.0,
                "19": 3.0,
                "20": 3.0
            }
        },
        "14-02-2025 Viernes": {
            "T8d": [
                {
                    "Hora_entrada": 7,
                    "Hora_salida": 15,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 8,
                    "Hora_salida": 16,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 10,
                    "Hora_salida": 18,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 11,
                    "Hora_salida": 19,
                    "empleados": 1,
                    "empleado": "Vendedores"
                }
            ],
            "T7m": [
                {
                    "Hora_entrada": 13,
                    "Hora_salida": 20,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 14,
                    "Hora_salida": 21,
                    "empleados": 2,
                    "empleado": "Vendedores"
                }
            ],
            "T6n": [],
            "Personal_necesario": {
                "7": 1,
                "8": 1,
                "9": 4,
                "10": 4,
                "11": 4,
                "12": 5,
                "13": 5,
                "14": 5,
                "15": 4,
                "16": 5,
                "17": 5,
                "18": 4,
                "19": 3,
                "20": 2
            },
            "Empleados_asignados": {
                "7": 1.0,
                "8": 2.0,
                "9": 2.0,
                "10": 3.0,
                "11": 4.0,
                "12": 4.0,
                "13": 5.0,
                "14": 7.0,
                "15": 6.0,
                "16": 5.0,
                "17": 5.0,
                "18": 4.0,
                "19": 3.0,
                "20": 2.0
            }
        },
        "15-02-2025 S\u00e1bado": {
            "T8d": [
                {
                    "Hora_entrada": 7,
                    "Hora_salida": 15,
                    "empleados": 2,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 8,
                    "Hora_salida": 16,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 9,
                    "Hora_salida": 17,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 11,
                    "Hora_salida": 19,
                    "empleados": 1,
                    "empleado": "Vendedores"
                }
            ],
            "T7m": [
                {
                    "Hora_entrada": 12,
                    "Hora_salida": 19,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 14,
                    "Hora_salida": 21,
                    "empleados": 2,
                    "empleado": "Vendedores"
                }
            ],
            "T6n": [],
            "Personal_necesario": {
                "7": 2,
                "8": 2,
                "9": 4,
                "10": 4,
                "11": 5,
                "12": 6,
                "13": 6,
                "14": 5,
                "15": 5,
                "16": 5,
                "17": 4,
                "18": 4,
                "19": 4,
                "20": 2
            },
            "Empleados_asignados": {
                "7": 2.0,
                "8": 3.0,
                "9": 4.0,
                "10": 4.0,
                "11": 5.0,
                "12": 6.0,
                "13": 6.0,
                "14": 8.0,
                "15": 6.0,
                "16": 5.0,
                "17": 4.0,
                "18": 4.0,
                "19": 2.0,
                "20": 2.0
            }
        },
        "16-02-2025 Domingo": {
            "T8d": [
                {
                    "Hora_entrada": 7,
                    "Hora_salida": 15,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 8,
                    "Hora_salida": 16,
                    "empleados": 1,
                    "empleado": "Vendedores"
                },
                {
                    "Hora_entrada": 10,
                    "Hora_salida": 18,
                    "empleados": 2,
                    "empleado": "Vendedores"
                }
            ],
            "T7m": [
                {
                    "Hora_entrada": 14,
                    "Hora_salida": 21,
                    "empleados": 2,
                    "empleado": "Vendedores"
                }
            ],
            "T6n": [],
            "Personal_necesario": {
                "7": 1,
                "8": 1,
                "9": 3,
                "10": 4,
                "11": 4,
                "12": 4,
                "13": 4,
                "14": 4,
                "15": 5,
                "16": 4,
                "17": 4,
                "18": 4,
                "19": 2,
                "20": 1
            },
            "Empleados_asignados": {
                "7": 1.0,
                "8": 2.0,
                "9": 2.0,
                "10": 4.0,
                "11": 4.0,
                "12": 4.0,
                "13": 4.0,
                "14": 6.0,
                "15": 5.0,
                "16": 4.0,
                "17": 4.0,
                "18": 2.0,
                "19": 2.0,
                "20": 2.0
            }
        }
    }
}
asignaciones = optimizar_horarios(data)

print(json.dumps(asignaciones, indent=4, ensure_ascii=False))

/Users/gabrielchavez/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pulp/pulp.py:1657: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/gabrielchavez/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/5s/y64xy8cd6cd355kr26pngsrh0000gn/T/44e23a4d53c24de0b89e1304e594c793-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/5s/y64xy8cd6cd355kr26pngsrh0000gn/T/44e23a4d53c24de0b89e1304e594c793-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 258 RHS
At line 268 BOUNDS
At line 303 ENDATA
Problem MODEL has 9 rows, 34 columns and 174 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -1 to -1.79769e+308
Probing was tried 0 times and created 0